In [1]:
!pip -q install -U langgraph openai pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.


# Global Configs

In [2]:
from google.colab import userdata

WOLFRAM_APP_ID = userdata.get('WOLFRAM_APP_ID')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-5.2"
SERVICE_TIER = "flex"
LANGUAGE = 'Ukrainian'

# Shared Components

## Prompts

In [3]:
TRANSLATION_SYSTEM = """You are a professional mathematical translator. Translate the given mathematical content into {language}.

Rules:
- Preserve ALL LaTeX notation exactly as-is (do not translate math expressions)
- Translate only the natural language portions
- Maintain the same markdown formatting
- Keep mathematical terms precise and use standard terminology in the target language
- Do not add or remove any content
"""

In [4]:
TRANSLATION_USER = """Translate the following mathematical content into {language}:

{content}"""

In [5]:
WOLFRAM_TO_LATEX_SYSTEM = """You are a mathematical notation converter. Convert Wolfram|Alpha plaintext output to clean LaTeX.

Rules:
- Output ONLY the LaTeX expression, nothing else
- Use proper LaTeX commands: \\frac{}{}, \\sqrt{}, \\int, \\sum, etc.
- Use \\ln for natural log (not \\log)
- Replace "constant" with "+ C" for indefinite integrals
- Handle multiple solutions by separating with \\\\ (newline in LaTeX)
- Do not include equation numbers or labels
- Do not wrap in $ or $$ delimiters
"""

WOLFRAM_TO_LATEX_USER = """Convert this Wolfram|Alpha result to LaTeX:

{plaintext}"""

## Models

In [6]:
from pydantic import BaseModel, Field
from typing import List, Literal, TypedDict

In [7]:
class Problem(BaseModel):
    """A generated math problem."""
    statement_latex: str = Field(description="Full problem statement (natural language + LaTeX allowed).")
    domain: Literal["calculus", "linear_algebra"]
    topic: str
    difficulty: Literal["easy", "medium", "hard"]
    tags: List[str] = Field(default_factory=list)

In [8]:
class Equation(BaseModel):
    """A single practice equation, compatible with Problem for solving."""
    statement_latex: str = Field(description="Instruction + equation, e.g., 'Compute: $\\int x^2 \\, dx$'")
    wolfram_query: str = Field(description="Wolfram|Alpha query for this equation, e.g., 'integrate x^2 dx'")
    tags: List[str] = Field(default_factory=list)

## Utilities

In [9]:
import random
import re
import time
import requests
from pydantic import ValidationError
from langgraph.graph import START, END, StateGraph
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI(api_key=OPENAI_API_KEY)

In [10]:
def wolfram_to_latex(plaintext: str, token_usage: dict = None, usage_log: list = None) -> dict:
    """
    Convert Wolfram|Alpha plaintext to LaTeX using LLM.

    Returns dict with 'latex', 'token_usage', 'usage_log'
    """
    if not plaintext:
        return {
            "latex": "",
            "token_usage": token_usage or {"input": 0, "output": 0, "total": 0},
            "usage_log": usage_log or [],
        }

    token_usage = dict(token_usage or {"input": 0, "output": 0, "total": 0})
    usage_log = list(usage_log or [])

    try:
        resp = client.chat.completions.create(
            model=MODEL,
            service_tier=SERVICE_TIER,
            messages=[
                {"role": "system", "content": WOLFRAM_TO_LATEX_SYSTEM},
                {"role": "user", "content": WOLFRAM_TO_LATEX_USER.format(plaintext=plaintext)},
            ],
            temperature=0.0,
            max_completion_tokens=500,
        )

        latex_result = resp.choices[0].message.content.strip()

        # Clean up any accidental delimiters
        latex_result = latex_result.strip('$').strip()

        usage = getattr(resp, "usage", None)
        if usage is not None:
            token_usage["input"] += int(getattr(usage, "prompt_tokens", 0))
            token_usage["output"] += int(getattr(usage, "completion_tokens", 0))
            token_usage["total"] += int(getattr(usage, "total_tokens", 0))
            usage_log.append({
                "stage": "wolfram_to_latex",
                "input_tokens": int(getattr(usage, "prompt_tokens", 0)),
                "output_tokens": int(getattr(usage, "completion_tokens", 0)),
                "total_tokens": int(getattr(usage, "total_tokens", 0)),
            })

        return {
            "latex": latex_result,
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

    except Exception as e:
        print(f"[wolfram_to_latex] Error: {e}")
        return {
            "latex": "",
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

In [11]:
def robust_parse(
    client,
    instructions: str,
    prompt: str,
    response_format,
    temperature: float = 0.5,
    max_completion_tokens: int = 1500,
    max_retries: int = 3,
):
    """
    Wrapper for structured output parsing with retry logic.
    Handles truncation, validation and API errors.
    """
    last_error = None

    for attempt in range(max_retries):
        try:
            adjusted_tokens = max_completion_tokens + (attempt * 500)

            resp = client.responses.parse(
                model=MODEL,
                service_tier=SERVICE_TIER,
                instructions=instructions,
                input=[{"role": "user", "content": prompt}],
                text_format=response_format,
                temperature=temperature,
                max_output_tokens=adjusted_tokens,
            )

            return {
                "parsed": resp.output_parsed,
                "usage": getattr(resp, "usage", None),
                "attempts": attempt + 1,
            }

        except ValidationError as e:
            last_error = e
            print(f"[Attempt {attempt + 1}/{max_retries}] Validation error: {str(e)[:100]}...")
            time.sleep(0.5)
            temperature = max(0.2, temperature - 0.1)

        except Exception as e:
            last_error = e
            print(f"[Attempt {attempt + 1}/{max_retries}] Unexpected error: {str(e)[:100]}...")
            time.sleep(0.5)

    raise RuntimeError(f"Failed after {max_retries} attempts. Last error: {last_error}")

In [12]:
def translate_text(content: str, target_language: str, token_usage: dict = None, usage_log: list = None) -> dict:
    """
    Translate mathematical content to the target language.
    Preserves LaTeX notation and markdown formatting.

    Returns dict with 'translated', 'token_usage', 'usage_log'
    """
    if not target_language or target_language.lower() == "english":
        return {
            "translated": content,
            "token_usage": token_usage or {"input": 0, "output": 0, "total": 0},
            "usage_log": usage_log or [],
        }

    token_usage = dict(token_usage or {"input": 0, "output": 0, "total": 0})
    usage_log = list(usage_log or [])

    try:
        resp = client.chat.completions.create(
            model=MODEL,
            service_tier=SERVICE_TIER,
            messages=[
                {"role": "system", "content": TRANSLATION_SYSTEM.format(language=target_language)},
                {"role": "user", "content": TRANSLATION_USER.format(language=target_language, content=content)},
            ],
            temperature=0.3,
            max_completion_tokens=2000,
        )

        translated = resp.choices[0].message.content

        usage = getattr(resp, "usage", None)
        if usage is not None:
            token_usage["input"] += int(getattr(usage, "prompt_tokens", 0))
            token_usage["output"] += int(getattr(usage, "completion_tokens", 0))
            token_usage["total"] += int(getattr(usage, "total_tokens", 0))
            usage_log.append({
                "stage": f"translate_to_{target_language}",
                "input_tokens": int(getattr(usage, "prompt_tokens", 0)),
                "output_tokens": int(getattr(usage, "completion_tokens", 0)),
                "total_tokens": int(getattr(usage, "total_tokens", 0)),
            })

        return {
            "translated": translated,
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

    except Exception as e:
        print(f"[translate_text] Error: {e}")
        return {
            "translated": content,  # Fallback to original
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

In [13]:
def render_latex_for_markdown(text: str) -> str:
    """
    Convert LaTeX delimiters to markdown-compatible format.
    Converts: \\[...\\] → $$...$$ and \\(...\\) → $...$
    """
    text = re.sub(r"\\\[\s*", "$$\n", text)
    text = re.sub(r"\s*\\\]", "\n$$", text)
    text = re.sub(r"\\\(", "$", text)
    text = re.sub(r"\\\)", "$", text)
    return text


def print_problem(s: str) -> None:
    display(Markdown(render_latex_for_markdown(s)))


def print_solution(s: str) -> None:
    display(Markdown(render_latex_for_markdown(s)))

In [14]:
def format_token_summary(token_usage: dict) -> str:
    """Format token usage as consistent string."""
    return (
        f"Input: {token_usage.get('input', 0)} | "
        f"Output: {token_usage.get('output', 0)} | "
        f"Total: {token_usage.get('total', 0)}"
    )

---
# Equation Generator Agent

## Models

In [15]:
class EquationSkeletonItem(BaseModel):
    structure: str
    technique: str
    complexity: str

In [16]:
class EquationSkeletonList(BaseModel):
    items: List[EquationSkeletonItem]

In [17]:
class EquationGeneratorState(TypedDict, total=False):
    # Input
    domain: str
    topic: str
    difficulty: str
    target_language: str

    # Skeleton stage
    equation_skeletons: list[dict]
    selected_skeleton: dict

    # Generation
    attempts: int
    errors: list[str]

    # Output
    equation: Equation
    equation_markdown: str
    equation_markdown_translated: str

    # Tracking
    token_usage: dict
    usage_log: list[dict]

## Prompts

In [18]:
EQUATION_SYSTEM_PROMPT = """You are a math drill generator specializing in creating practice equations.
Generate ONE concise, one-liner equation for students to practice raw calculation skills.

Rules:
- The equation must be self-contained and solvable
- Combine instruction and equation into a single statement
- Format: "Instruction: $latex$" (e.g., "Compute: $\\int x^2 \\, dx$")
- Instruction should be ONE word or short phrase: "Compute", "Solve", "Evaluate", "Find", "Simplify", "Prove that", etc.
- Use $...$ delimiters for LaTeX
- Do NOT include solutions, answers, or hints
- Provide a Wolfram|Alpha query that will solve this equation (plain text, not LaTeX)
"""

EQUATION_GENERATION_USER = """Generate ONE practice equation based on this concept:

Domain: {domain}
Topic: "{topic}"
Difficulty: {difficulty}

**Equation concept to develop:**
- Structure: {structure}
- Technique: {technique}
- Complexity note: {complexity}

Requirements:
- Single one-liner in format: "Instruction: $latex$"
- Example: "Compute: $\\int \\frac{{2x}}{{1+x^2}} \\, dx$"
- Instruction should be minimal: "Compute", "Solve", "Evaluate", "Simplify", "Find", or "Prove that"
- Include a wolfram_query field with the plain-text Wolfram|Alpha query (e.g., "integrate 2x/(1+x^2) dx")

Return a JSON object with statement_latex, wolfram_query, and tags."""

EQUATION_SKELETON_SYSTEM = """You are a math curriculum designer. Generate DIVERSE equation concepts for practice drills.
Focus on varying the mathematical structures, techniques required, and complexity."""

EQUATION_SKELETON_USER = """For the topic "{topic}" in {domain} at {difficulty} level, propose {n} DIVERSE equation/expression types.

Examples of variation:
- Different function types (polynomial, rational, trigonometric, exponential, logarithmic)
- Different techniques (substitution, parts, partial fractions, trig identities)
- Different structures (definite vs indefinite, single vs nested, with parameters)

Return a JSON array of {n} objects with:
- structure: brief description of the equation structure (e.g., "rational function with quadratic denominator")
- technique: primary technique needed (e.g., "partial fractions")
- complexity: what makes it interesting at this difficulty level
"""

## Nodes

In [19]:
def generate_equation_skeletons(state: EquationGeneratorState) -> dict:
    """Generate diverse equation structure concepts, then randomly select one."""
    topic = state.get("topic", "").strip()
    domain = state.get("domain", "calculus")
    difficulty = state.get("difficulty", "medium")

    prompt = EQUATION_SKELETON_USER.format(
        topic=topic, domain=domain, difficulty=difficulty, n=5
    )

    result = robust_parse(
        client=client,
        instructions=EQUATION_SKELETON_SYSTEM,
        prompt=prompt,
        response_format=EquationSkeletonList,
        temperature=1.0,
        max_completion_tokens=600,
        max_retries=3,
    )

    skeletons = [s.model_dump() for s in result["parsed"].items]
    selected = random.choice(skeletons)

    usage = result["usage"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if usage is not None:
        token_usage["input"] += int(getattr(usage, "input_tokens", 0))
        token_usage["output"] += int(getattr(usage, "output_tokens", 0))
        token_usage["total"] += int(getattr(usage, "total_tokens", 0))
        usage_log.append({
            "stage": "equation_skeletons",
            "attempts": result["attempts"],
            "input_tokens": int(getattr(usage, "input_tokens", 0)),
            "output_tokens": int(getattr(usage, "output_tokens", 0)),
            "total_tokens": int(getattr(usage, "total_tokens", 0)),
        })

    return {
        "equation_skeletons": skeletons,
        "selected_skeleton": selected,
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [20]:
def generate_equation(state: EquationGeneratorState) -> dict:
    """Generate a single equation based on selected skeleton."""
    topic = state.get("topic", "").strip()
    domain = state.get("domain", "calculus")
    difficulty = state.get("difficulty", "medium")
    skeleton = state["selected_skeleton"]

    prompt = EQUATION_GENERATION_USER.format(
        domain=domain,
        topic=topic,
        difficulty=difficulty,
        structure=skeleton["structure"],
        technique=skeleton["technique"],
        complexity=skeleton["complexity"],
    )

    result = robust_parse(
        client=client,
        instructions=EQUATION_SYSTEM_PROMPT,
        prompt=prompt,
        response_format=Equation,
        temperature=0.8,
        max_completion_tokens=300,
        max_retries=3,
    )

    equation = result["parsed"]

    usage = result["usage"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if usage is not None:
        token_usage["input"] += int(getattr(usage, "input_tokens", 0))
        token_usage["output"] += int(getattr(usage, "output_tokens", 0))
        token_usage["total"] += int(getattr(usage, "total_tokens", 0))
        usage_log.append({
            "stage": "generate_equation",
            "attempts": state.get("attempts", 0) + 1,
            "input_tokens": int(getattr(usage, "input_tokens", 0)),
            "output_tokens": int(getattr(usage, "output_tokens", 0)),
            "total_tokens": int(getattr(usage, "total_tokens", 0)),
        })

    return {
        "equation": equation,
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [21]:
def validate_equation(state: EquationGeneratorState) -> dict:
    """Validate the generated equation."""
    eq = state.get("equation")
    errors = []

    if not eq:
        errors.append("No equation generated.")
        return {"errors": errors, "attempts": state.get("attempts", 0) + 1}

    if not eq.statement_latex:
        errors.append("Missing statement_latex.")
    elif "$" not in eq.statement_latex:
        errors.append("Statement should contain LaTeX with $...$ delimiters.")
    elif ":" not in eq.statement_latex:
        errors.append("Statement should have format 'Instruction: $latex$'.")

    solution_patterns = [r"=\s*-?\d+\.?\d*\s*\$", r"\banswer\b", r"\bsolution\b"]
    for pattern in solution_patterns:
        if re.search(pattern, eq.statement_latex, re.IGNORECASE):
            errors.append("Equation may contain an answer/solution.")
            break

    return {
        "errors": errors,
        "attempts": state.get("attempts", 0) + 1,
    }

In [22]:
def finalize_equation(state: EquationGeneratorState) -> dict:
    """Render equation as markdown and translate if needed."""
    eq = state["equation"]
    target_language = state.get("target_language", "")

    # statement_latex is already the complete output
    equation_md = eq.statement_latex

    # Translate if needed
    result = translate_text(
        content=equation_md,
        target_language=target_language,
        token_usage=state.get("token_usage"),
        usage_log=state.get("usage_log"),
    )

    return {
        "equation_markdown": equation_md,
        "equation_markdown_translated": result["translated"],
        "token_usage": result["token_usage"],
        "usage_log": result["usage_log"],
    }

## Edges

In [23]:
def route_after_equation_validate(state: EquationGeneratorState) -> str:
    """Route based on validation. Retry if errors and under limit."""
    if state.get("errors") and state.get("attempts", 0) < 3:
        return "retry"
    return "done"

## Compilation

In [24]:
equation_graph = StateGraph(EquationGeneratorState)

equation_graph.add_node("generate_equation_skeletons", generate_equation_skeletons)
equation_graph.add_node("generate_equation", generate_equation)
equation_graph.add_node("validate_equation", validate_equation)
equation_graph.add_node("finalize_equation", finalize_equation)

equation_graph.add_edge(START, "generate_equation_skeletons")
equation_graph.add_edge("generate_equation_skeletons", "generate_equation")
equation_graph.add_edge("generate_equation", "validate_equation")

equation_graph.add_conditional_edges(
    "validate_equation",
    route_after_equation_validate,
    {"retry": "generate_equation", "done": "finalize_equation"},
)

equation_graph.add_edge("finalize_equation", END)

equation_generator = equation_graph.compile()

---
# Problem Generator Agent

## Prompts

In [25]:
GENERATOR_SYSTEM_PROMPT = """You are Math Tutor: a college-level math PROBLEM GENERATOR.
Generate exactly ONE practice problem ONLY.

Hard constraints:
- Do NOT include any solution steps, hints that reveal the solution, or final answers.
- Output MUST match the JSON schema exactly.
- statement_latex may include natural language + LaTeX math blocks.
- Use $...$ for inline math and $$...$$ for display math.
"""

SKELETON_SYSTEM = """You are a creative math curriculum designer. Your job is to propose DIVERSE problem concepts—not full problems, just ideas that vary in structure, context, and required techniques."""

SKELETON_USER_TEMPLATE = """For the topic "{topic}" in {domain} at {difficulty} level, propose {n} VERY DIFFERENT problem concepts.

Vary across these dimensions:
- Problem type (computational, proof, conceptual, construction, counterexample, application)
- Mathematical objects (functions, sequences, matrices, geometric shapes, etc.)
- Context (pure math, physics, economics, geometry, probability)
- Required technique or insight

Return a JSON array of {n} objects with:
- concept: one-sentence description of the problem idea
- problem_type: one of [computational, proof, conceptual, construction, counterexample, application, multi-part]
- key_twist: what makes this problem interesting or non-standard
"""

GENERATION_USER_TEMPLATE = """Generate 1 practice problem based on this concept:

Domain: {domain}
Topic: "{topic}"
Difficulty: {difficulty}

**Problem concept to develop:**
{concept}

**Problem type:** {problem_type}
**Key twist to incorporate:** {key_twist}

Expand this concept into a complete, well-posed problem. Return ONLY the JSON object."""

## State

In [26]:
class GeneratorState(TypedDict, total=False):
    topic: str
    domain: str
    difficulty: str
    target_language: str

    skeletons: list[dict]
    selected_skeleton: dict

    prompt: str
    attempts: int
    errors: list[str]

    problem: Problem
    problem_markdown: str
    problem_markdown_translated: str

    token_usage: dict
    usage_log: list[dict]

## Models

In [27]:
class SkeletonItem(BaseModel):
    concept: str
    problem_type: str
    key_twist: str


class SkeletonList(BaseModel):
    items: List[SkeletonItem]

## Nodes

In [28]:
def generate_skeletons(state: GeneratorState) -> dict:
    """Generates a diverse set of problem concepts (skeletons). Randomly selects one."""
    topic = state["topic"].strip()
    domain = state.get("domain", "calculus")
    difficulty = state.get("difficulty", "medium")

    prompt = SKELETON_USER_TEMPLATE.format(
        topic=topic, domain=domain, difficulty=difficulty, n=5,
    )

    result = robust_parse(
        client=client,
        instructions=SKELETON_SYSTEM,
        prompt=prompt,
        response_format=SkeletonList,
        temperature=1.1,
        max_completion_tokens=800,
        max_retries=3,
    )

    skeletons = [s.model_dump() for s in result["parsed"].items]
    selected = random.choice(skeletons)

    usage = result["usage"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if usage is not None:
        token_usage["input"] += int(getattr(usage, "input_tokens", 0))
        token_usage["output"] += int(getattr(usage, "output_tokens", 0))
        token_usage["total"] += int(getattr(usage, "total_tokens", 0))
        usage_log.append({
            "stage": "skeleton",
            "attempts": result["attempts"],
            "input_tokens": int(getattr(usage, "input_tokens", 0)),
            "output_tokens": int(getattr(usage, "output_tokens", 0)),
            "total_tokens": int(getattr(usage, "total_tokens", 0)),
        })

    return {
        "skeletons": skeletons,
        "selected_skeleton": selected,
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [29]:
def build_prompt(state: GeneratorState) -> dict:
    """Builds the problem generation prompt using the selected skeleton."""
    topic = state["topic"].strip()
    domain = state.get("domain", "calculus")
    difficulty = state.get("difficulty", "medium")
    skeleton = state["selected_skeleton"]

    prompt = GENERATION_USER_TEMPLATE.format(
        domain=domain,
        topic=topic,
        difficulty=difficulty,
        concept=skeleton["concept"],
        problem_type=skeleton["problem_type"],
        key_twist=skeleton["key_twist"],
    )

    return {
        "topic": topic,
        "domain": domain,
        "difficulty": difficulty,
        "prompt": prompt,
        "attempts": state.get("attempts", 0),
        "errors": [],
        "token_usage": state.get("token_usage", {"input": 0, "output": 0, "total": 0}),
        "usage_log": state.get("usage_log", []),
    }

In [30]:
def generate_problem(state: GeneratorState) -> dict:
    """Takes a skeleton-adjusted prompt and generates a problem."""
    result = robust_parse(
        client=client,
        instructions=GENERATOR_SYSTEM_PROMPT,
        prompt=state["prompt"],
        response_format=Problem,
        temperature=0.9,
        max_completion_tokens=1500,
        max_retries=3,
    )

    problem: Problem = result["parsed"]

    usage = result["usage"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if usage is not None:
        token_usage["input"] += int(getattr(usage, "input_tokens", 0))
        token_usage["output"] += int(getattr(usage, "output_tokens", 0))
        token_usage["total"] += int(getattr(usage, "total_tokens", 0))
        usage_log.append({
            "attempt": state.get("attempts", 0) + 1,
            "parse_attempts": result["attempts"],
            "input_tokens": int(getattr(usage, "input_tokens", 0)),
            "output_tokens": int(getattr(usage, "output_tokens", 0)),
            "total_tokens": int(getattr(usage, "total_tokens", 0)),
        })

    return {"problem": problem, "token_usage": token_usage, "usage_log": usage_log}

In [31]:
def validate_problem(state: GeneratorState) -> dict:
    """Rule-based validation of the generated problem."""
    p: Problem = state["problem"]
    errors = []

    if state["topic"].lower() not in p.topic.lower():
        errors.append(f"Returned topic '{p.topic}' drifted from requested '{state['topic']}'.")

    if p.domain != state["domain"]:
        errors.append(f"Returned domain '{p.domain}' != requested '{state['domain']}'.")
    if p.difficulty != state["difficulty"]:
        errors.append(f"Returned difficulty '{p.difficulty}' != requested '{state['difficulty']}'.")

    attempts = state.get("attempts", 0) + 1
    return {"errors": errors, "attempts": attempts}

In [32]:
def finalize_problem(state: GeneratorState) -> dict:
    """Finalizes the problem: renders markdown and translates if needed."""
    problem_md = render_latex_for_markdown(state["problem"].statement_latex)
    target_language = state.get("target_language", "")

    # Translate if target language specified
    result = translate_text(
        content=problem_md,
        target_language=target_language,
        token_usage=state.get("token_usage"),
        usage_log=state.get("usage_log"),
    )

    return {
        "problem_markdown": problem_md,
        "problem_markdown_translated": result["translated"],
        "token_usage": result["token_usage"],
        "usage_log": result["usage_log"],
    }

## Edges

In [33]:
def route_after_problem_validate(state: GeneratorState) -> str:
    """Routes based on validation results. Retries generation if errors found."""
    if state.get("errors") and state.get("attempts", 0) < 3:
        return "retry"
    return "done"

## Compilation

In [34]:
generator_graph = StateGraph(GeneratorState)

generator_graph.add_node("generate_skeletons", generate_skeletons)
generator_graph.add_node("build_prompt", build_prompt)
generator_graph.add_node("generate_problem", generate_problem)
generator_graph.add_node("validate_problem", validate_problem)
generator_graph.add_node("finalize_problem", finalize_problem)

generator_graph.add_edge(START, "generate_skeletons")
generator_graph.add_edge("generate_skeletons", "build_prompt")
generator_graph.add_edge("build_prompt", "generate_problem")
generator_graph.add_edge("generate_problem", "validate_problem")

generator_graph.add_conditional_edges(
    "validate_problem",
    route_after_problem_validate,
    {"retry": "generate_problem", "done": "finalize_problem"},
)

generator_graph.add_edge("finalize_problem", END)

problem_generator = generator_graph.compile()

---
# Problem Solver Agent

## Configs

In [35]:
WA_ENDPOINT = "https://api.wolframalpha.com/v2/query"
MAX_PLAN_STEPS = 10
MAX_STEP_SIZE_TOKENS = 200
MAX_STEP_WOLFRAM_SIZE = 200

## Prompts

In [36]:
SOLUTION_PLANNER_SYSTEM = """You are an expert math problem solver. Given a problem, create a step-by-step solution PLAN.

For each step, specify:
- step_number: integer
- description: what this step accomplishes
- step_type: one of ["reasoning", "symbolic_compute", "verify"]
  - "reasoning": logical deduction, setting up equations, interpreting results
  - "symbolic_compute": integration, differentiation, solving equations, simplification (use Wolfram)
  - "verify": checking a result is correct
- depends_on: list of step numbers this depends on (empty for first steps)
- wolfram_query: if step_type is "symbolic_compute", provide the Wolfram|Alpha query (plain text, not LaTeX)

Think carefully about what requires symbolic computation vs. pure reasoning."""

SOLUTION_PLANNER_USER = """Create a solution plan for this problem:

{problem_statement}

Break it into atomic steps. For any symbolic computation (integrals, derivatives, solving equations, simplification), mark as "symbolic_compute" and provide the Wolfram query.

IMPORTANT: Keep the plan concise. Maximum {max_steps} steps. Keep descriptions brief - under {max_tokens} tokens.
"""

STEP_EXECUTOR_SYSTEM = """You are executing ONE step of a math solution. You have context from previous steps.

Write clear, detailed mathematical reasoning for this step. Use LaTeX notation ($...$ for inline, $$...$$ for display).
If Wolfram|Alpha results are provided, interpret and explain them.
Be precise and show your work."""

STEP_EXECUTOR_USER = """## Problem
{problem_statement}

## Current Step
**Step {step_number}:** {step_description}

## Previous Steps Completed
{previous_steps}

## Wolfram|Alpha Result (if applicable)
{wolfram_result}

Execute this step. Write clear mathematical reasoning with LaTeX notation."""

SOLUTION_COMPILER_SYSTEM = """You are compiling individual solution steps into a polished, coherent final solution.
- Maintain logical flow between steps
- Use consistent LaTeX notation ($...$ for inline, $$...$$ for display)
- Ensure the final answer is clearly stated
- Format as clean Markdown with proper math blocks"""

SOLUTION_COMPILER_USER = """## Problem
{problem_statement}

## Executed Steps
{all_steps}

Compile these into a single, polished step-by-step solution. Ensure smooth transitions and a clear final answer."""

## State

In [37]:
class SolverState(TypedDict, total=False):
    # Input
    problem: Problem | Equation
    target_language: str
    step_by_step: bool

    # Planning
    solution_plan: list[dict]
    current_step_index: int
    step_results: list[dict]

    # Current solution attempt
    solution: str
    solution_errors: list[str]
    solution_attempts: int

    # Best solution tracking
    best_solution: str
    best_solution_errors: list[str]
    best_solution_step_results: list[dict]

    # WA tracking
    wa_calls: int

    # Token tracking
    token_usage: dict
    usage_log: list[dict]

    # Output
    solution_markdown: str
    solution_markdown_translated: str
    summary: str

## Models

In [38]:
class SolutionStep(BaseModel):
    step_number: int
    description: str = Field(max_length=MAX_STEP_SIZE_TOKENS)
    step_type: Literal["reasoning", "symbolic_compute", "verify"]
    depends_on: List[int] = Field(default_factory=list)
    wolfram_query: str = Field(default="", max_length=MAX_STEP_WOLFRAM_SIZE)


class SolutionPlan(BaseModel):
    steps: List[SolutionStep] = Field(max_length=MAX_PLAN_STEPS)

## Utilities

In [39]:
def wolfram_solve(query: str, appid: str, timeout: int = 30) -> dict:
    """Query Wolfram|Alpha and extract computational results."""
    params = {
        "input": query,
        "appid": appid,
        "output": "json",
        "format": "plaintext",
    }

    try:
        response = requests.get(WA_ENDPOINT, params=params, timeout=timeout)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        return {
            "ok": False,
            "result": "",
            "pods": [],
            "raw": {},
        }

    query_result = data.get("queryresult", {})

    if not query_result.get("success", False):
        return {
            "ok": False,
            "result": "",
            "pods": [],
            "raw": data,
        }

    pods = query_result.get("pods", []) or []
    priority_titles = [
        "result", "results", "solution", "solutions",
        "definite integral", "indefinite integral",
        "derivative", "limit", "sum", "root", "roots", "exact result",
    ]

    extracted_pods = []
    primary_result = ""

    for pod in pods:
        title = pod.get("title", "")
        pod_text = _extract_pod_text(pod)

        if not pod_text:
            continue

        extracted_pods.append({"title": title, "text": pod_text})

        if not primary_result and title.lower() in priority_titles:
            primary_result = pod_text

    if not primary_result and extracted_pods:
        primary_result = extracted_pods[0]["text"]

    return {
        "ok": True,
        "result": primary_result,
        "pods": extracted_pods,
        "raw": data,
    }


def _extract_pod_text(pod: dict) -> str:
    """Extract plaintext content from a Wolfram|Alpha pod."""
    parts = []
    for subpod in pod.get("subpods", []) or []:
        text = (subpod.get("plaintext") or "").strip()
        if text:
            parts.append(text)
    return "\n".join(parts).strip()

In [40]:
def format_steps_as_solution(step_results: list[dict], header: str = "## Solution") -> str:
    """Format executed step results into markdown solution text."""
    if not step_results:
        return ""

    parts = [header, ""]
    for sr in step_results:
        parts.append(f"### Step {sr['step_number']}: {sr['description']}\n")
        if sr.get("wolfram_result"):
            parts.append(f"**Computation:**\n```\n{sr['wolfram_result']}\n```\n")
        reasoning = render_latex_for_markdown(sr['reasoning'])
        parts.append(f"{reasoning}\n")

    return "\n".join(parts)


def format_plan_as_outline(plan: list[dict], header: str = "## Solution Outline") -> str:
    """Format solution plan as a fallback outline when steps weren't executed."""
    if not plan:
        return "_No solution plan available._"

    parts = [header, "", "_Full solution could not be generated. Here is the planned approach:_", ""]
    for step in plan:
        step_line = f"**Step {step['step_number']} ({step['step_type']}):** {step['description']}"
        parts.append(step_line)
        if step.get("wolfram_query"):
            parts.append(f"  - Wolfram query: `{step['wolfram_query']}`")
        parts.append("")

    return "\n".join(parts)

## Nodes

In [41]:
def quick_solve(state: SolverState) -> dict:
    """Solve directly using Wolfram|Alpha without step-by-step explanation."""
    problem = state["problem"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    wolfram_query = getattr(problem, "wolfram_query", None)

    if not wolfram_query:
        statement = problem.statement_latex
        for prefix in ["Compute:", "Solve:", "Evaluate:", "Find:", "Simplify:", "Calculate:"]:
            if statement.startswith(prefix):
                statement = statement[len(prefix):].strip()
                break
        wolfram_query = statement.replace("$", "").replace("\\,", " ").replace("\\", " ").strip()

    wa_result = wolfram_solve(wolfram_query, WOLFRAM_APP_ID)
    wa_calls = 1

    if not wa_result.get("ok") or not wa_result.get("result"):
        return {
            "solution": "\n_Wolfram|Alpha could not solve this._",
            "best_solution": "\n_Wolfram|Alpha could not solve this._",
            "solution_errors": ["Wolfram|Alpha failed"],
            "best_solution_errors": ["Wolfram|Alpha failed"],
            "solution_attempts": 1,
            "wa_calls": wa_calls,
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

    conversion = wolfram_to_latex(
        plaintext=wa_result["result"],
        token_usage=token_usage,
        usage_log=usage_log,
    )

    latex_result = conversion["latex"]
    token_usage = conversion["token_usage"]
    usage_log = conversion["usage_log"]

    if latex_result:
        solution = f"\n${latex_result}$"
    else:
        solution = f"\n```\n{wa_result['result']}\n```"

    return {
        "solution": solution,
        "best_solution": solution,
        "solution_errors": [],
        "best_solution_errors": [],
        "solution_attempts": 1,
        "wa_calls": wa_calls,
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [42]:
def plan_solution(state: SolverState) -> dict:
    """Builds a step-by-step plan to solve the problem."""
    problem_text = state["problem"].statement_latex

    planning_prompt = SOLUTION_PLANNER_USER.format(problem_statement=problem_text, max_steps=MAX_PLAN_STEPS, max_tokens=MAX_STEP_SIZE_TOKENS)

    result = robust_parse(
        client=client,
        instructions=SOLUTION_PLANNER_SYSTEM,
        prompt=planning_prompt,
        response_format=SolutionPlan,
        temperature=0.4,
        max_completion_tokens=1500,
        max_retries=3,
    )

    plan: SolutionPlan = result["parsed"]
    steps = [s.model_dump() for s in plan.steps]

    usage = result["usage"]
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if usage is not None:
        token_usage["input"] += int(getattr(usage, "input_tokens", 0))
        token_usage["output"] += int(getattr(usage, "output_tokens", 0))
        token_usage["total"] += int(getattr(usage, "total_tokens", 0))
        usage_log.append({
            "stage": "plan_solution",
            "attempts": result["attempts"],
            "input_tokens": int(getattr(usage, "input_tokens", 0)),
            "output_tokens": int(getattr(usage, "output_tokens", 0)),
            "total_tokens": int(getattr(usage, "total_tokens", 0)),
        })

    return {
        "solution_plan": steps,
        "current_step_index": 0,
        "step_results": [],
        "wa_calls": state.get("wa_calls", 0),
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [43]:
def execute_step(state: SolverState) -> dict:
    """Executes a single step of the solution plan."""
    plan = state["solution_plan"]
    idx = state["current_step_index"]

    if idx >= len(plan):
        return {"current_step_index": idx}

    step = plan[idx]
    problem_text = state["problem"].statement_latex
    step_results = list(state.get("step_results", []))
    wa_calls = state.get("wa_calls", 0)
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    wolfram_result = ""

    try:
        if step["step_type"] == "symbolic_compute" and step.get("wolfram_query"):
            query = step["wolfram_query"].strip()
            if query and WOLFRAM_APP_ID:
                try:
                    wa_res = wolfram_solve(query, WOLFRAM_APP_ID)
                    wa_calls += 1
                    wolfram_result = wa_res.get("text", "") if wa_res.get("ok") else f"_Wolfram could not compute: {query}_"
                except Exception as e:
                    wolfram_result = f"_Wolfram API error: {str(e)[:100]}_"
                    wa_calls += 1

        if step_results:
            prev_parts = [f"**Step {sr['step_number']}:** {sr['description']}\n{sr['reasoning'][:1000]}..." for sr in step_results[-3:]]
            previous_steps_text = "\n\n".join(prev_parts)
        else:
            previous_steps_text = "_This is the first step._"

        prompt = STEP_EXECUTOR_USER.format(
            problem_statement=problem_text,
            step_number=step["step_number"],
            step_description=step["description"],
            previous_steps=previous_steps_text,
            wolfram_result=wolfram_result if wolfram_result else "_N/A - this is a reasoning step._",
        )

        resp = client.chat.completions.create(
            model=MODEL,
            service_tier=SERVICE_TIER,
            messages=[
                {"role": "system", "content": STEP_EXECUTOR_SYSTEM},
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
            max_completion_tokens=1200,
        )

        reasoning = resp.choices[0].message.content

        usage = getattr(resp, "usage", None)
        if usage is not None:
            token_usage["input"] += int(getattr(usage, "prompt_tokens", 0))
            token_usage["output"] += int(getattr(usage, "completion_tokens", 0))
            token_usage["total"] += int(getattr(usage, "total_tokens", 0))
            usage_log.append({
                "stage": f"execute_step_{step['step_number']}",
                "input_tokens": int(getattr(usage, "prompt_tokens", 0)),
                "output_tokens": int(getattr(usage, "completion_tokens", 0)),
                "total_tokens": int(getattr(usage, "total_tokens", 0)),
            })

    except Exception as e:
        print(f"[execute_step] Error on step {step['step_number']}: {e}")
        reasoning = f"_Step execution failed: {str(e)[:100]}_"

    step_results.append({
        "step_number": step["step_number"],
        "step_type": step["step_type"],
        "description": step["description"],
        "wolfram_query": step.get("wolfram_query", ""),
        "wolfram_result": wolfram_result,
        "reasoning": reasoning,
    })

    return {
        "step_results": step_results,
        "current_step_index": idx + 1,
        "wa_calls": wa_calls,
        "token_usage": token_usage,
        "usage_log": usage_log,
    }

In [44]:
def compile_solution(state: SolverState) -> dict:
    """Builds a polished solution from the executed steps."""
    problem_text = state["problem"].statement_latex
    step_results = state.get("step_results", [])
    token_usage = dict(state.get("token_usage", {"input": 0, "output": 0, "total": 0}))
    usage_log = list(state.get("usage_log", []))

    if not step_results:
        return {
            "solution": format_plan_as_outline(state.get("solution_plan", [])),
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

    all_steps_text = []
    for sr in step_results:
        step_text = f"### Step {sr['step_number']}: {sr['description']}\n\n"
        if sr.get("wolfram_result"):
            step_text += f"**Wolfram|Alpha computation:**\n{sr['wolfram_result']}\n\n"
        step_text += f"**Reasoning:**\n{sr['reasoning']}"
        all_steps_text.append(step_text)

    prompt = SOLUTION_COMPILER_USER.format(
        problem_statement=problem_text,
        all_steps="\n\n---\n\n".join(all_steps_text),
    )

    try:
        resp = client.chat.completions.create(
            model=MODEL,
            service_tier=SERVICE_TIER,
            messages=[
                {"role": "system", "content": SOLUTION_COMPILER_SYSTEM},
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
            max_completion_tokens=2500,
        )

        solution = resp.choices[0].message.content

        usage = getattr(resp, "usage", None)
        if usage is not None:
            token_usage["input"] += int(getattr(usage, "prompt_tokens", 0))
            token_usage["output"] += int(getattr(usage, "completion_tokens", 0))
            token_usage["total"] += int(getattr(usage, "total_tokens", 0))
            usage_log.append({
                "stage": "compile_solution",
                "input_tokens": int(getattr(usage, "prompt_tokens", 0)),
                "output_tokens": int(getattr(usage, "completion_tokens", 0)),
                "total_tokens": int(getattr(usage, "total_tokens", 0)),
            })

        return {"solution": solution, "token_usage": token_usage, "usage_log": usage_log}

    except Exception as e:
        print(f"[compile_solution] Error: {e}")
        return {
            "solution": format_steps_as_solution(step_results, "## Solution (Raw Steps)"),
            "token_usage": token_usage,
            "usage_log": usage_log,
        }

In [45]:
def validate_solution(state: SolverState) -> dict:
    """Validates the solution and tracks the best one."""
    solution = state.get("solution", "")
    errors = []

    if len(solution) < 200:
        errors.append("Solution is too short.")

    if "$" not in solution:
        errors.append("Solution lacks mathematical notation.")

    plan = state.get("solution_plan", [])
    results = state.get("step_results", [])
    if len(results) < len(plan):
        errors.append(f"Only {len(results)}/{len(plan)} steps were executed.")

    math_keywords = ["integral", "derivative", "=", "therefore", "thus", "hence", "solve", "compute"]
    has_math_content = any(kw in solution.lower() for kw in math_keywords)
    if not has_math_content:
        errors.append("Solution may lack mathematical reasoning.")

    # Track best solution
    current_solution = solution
    current_step_results = state.get("step_results", [])
    best_solution = state.get("best_solution", "")
    best_errors = state.get("best_solution_errors", None)
    best_step_results = state.get("best_solution_step_results", [])

    is_better = (
        best_errors is None or
        len(errors) < len(best_errors) or
        (len(errors) == len(best_errors) and len(current_solution) > len(best_solution))
    )

    if is_better and current_solution:
        best_solution = current_solution
        best_errors = errors
        best_step_results = current_step_results

    return {
        "solution_errors": errors,
        "solution_attempts": state.get("solution_attempts", 0) + 1,
        "best_solution": best_solution,
        "best_solution_errors": best_errors if best_errors is not None else errors,
        "best_solution_step_results": best_step_results,
    }

In [46]:
def finalize_solution(state: SolverState) -> dict:
    """Finalizes the output: renders markdown and translates if needed."""
    solution = state.get("best_solution") or state.get("solution", "")
    solution_errors = state.get("best_solution_errors", [])

    if not solution:
        step_results = state.get("best_solution_step_results") or state.get("step_results", [])
        if step_results:
            solution = format_steps_as_solution(step_results)
        else:
            solution = format_plan_as_outline(state.get("solution_plan", []))

    if solution_errors:
        disclaimer = (
            "\n\n---\n\n"
            "> **Note:** This solution may contain errors or incomplete reasoning. "
            "The following issues were detected:\n"
        )
        for err in solution_errors:
            disclaimer += f"> - {err}\n"
        disclaimer += ">\n> Please verify the steps independently."
        solution += disclaimer

    solution_md = render_latex_for_markdown(solution)

    # Translate if target language specified
    target_language = state.get("target_language", "")
    result = translate_text(
        content=solution_md,
        target_language=target_language,
        token_usage=state.get("token_usage"),
        usage_log=state.get("usage_log"),
    )

    tu = result["token_usage"]
    plan_len = len(state.get("solution_plan", []))
    executed = len(state.get("best_solution_step_results") or state.get("step_results", []))

    summary = (
        f"Solution attempts: {state.get('solution_attempts', 0)} | "
        f"Steps: {executed}/{plan_len} | "
        f"WA calls: {state.get('wa_calls', 0)} | "
        f"Errors: {len(solution_errors)} | "
        f"Input Tokens: {tu.get('input', 0)} "
        f"Output Tokens: {tu.get('output', 0)} "
        f"Total Tokens: {tu.get('total', 0)}"
    )

    if target_language:
        summary += f" | Language: {target_language}"

    return {
        "solution_markdown": solution_md,
        "solution_markdown_translated": result["translated"],
        "token_usage": result["token_usage"],
        "usage_log": result["usage_log"],
        "summary": summary,
    }

## Edges

In [47]:
def route_solver_mode(state: SolverState) -> str:
    """Route based on step_by_step flag."""
    if state.get("step_by_step", True):
        return "step_by_step"
    return "quick"

def route_step_execution(state: SolverState) -> str:
    """Loop controller. Checks if there are more steps to execute."""
    plan = state["solution_plan"]
    idx = state["current_step_index"]
    return "continue" if idx < len(plan) else "compile"


def route_after_solution_validate(state: SolverState) -> str:
    """Routes based on solution validation. Retries if errors found."""
    errors = state.get("solution_errors", [])
    attempts = state.get("solution_attempts", 0)

    if not errors:
        return "done"
    if attempts < 3:
        return "retry"
    return "done"

## Compilation

In [48]:
solver_graph = StateGraph(SolverState)

solver_graph.add_node("quick_solve", quick_solve)
solver_graph.add_node("plan_solution", plan_solution)
solver_graph.add_node("execute_step", execute_step)
solver_graph.add_node("compile_solution", compile_solution)
solver_graph.add_node("validate_solution", validate_solution)
solver_graph.add_node("finalize_solution", finalize_solution)

solver_graph.add_conditional_edges(
    START,
    route_solver_mode,
    {"step_by_step": "plan_solution", "quick": "quick_solve"},
)

# Quick solve path
solver_graph.add_edge("quick_solve", "finalize_solution")

# Step-by-step path
solver_graph.add_edge("plan_solution", "execute_step")

solver_graph.add_conditional_edges(
    "execute_step",
    route_step_execution,
    {"continue": "execute_step", "compile": "compile_solution"},
)

solver_graph.add_edge("compile_solution", "validate_solution")

solver_graph.add_conditional_edges(
    "validate_solution",
    route_after_solution_validate,
    {"retry": "plan_solution", "done": "finalize_solution"},
)

solver_graph.add_edge("finalize_solution", END)

problem_solver = solver_graph.compile()

---
# Invocation

In [49]:
# =====================================================
# CONFIGURATION
# =====================================================

# Mode: "equation" or "problem"
MODE = "problem"  # @param ["equation", "problem"]

# Solver mode: step-by-step explanation or quick WA answer
QUICK_SOLUTION = False  # @param {type:"boolean"}
QUICK_SOLUTION = QUICK_SOLUTION and MODE == 'equation' # allow quick solutions for Equations only

# Problem/Equation parameters
DOMAIN = "calculus" # @param ["calculus","linear_algebra","algebra"]
TOPIC = "Volume Calculation"  # @param {type:"string"}
DIFFICULTY = "medium"  # @param ["easy", "medium", "hard"]

# Language for output (empty or "English" = no translation)
LANGUAGE = "Ukrainian"  # @param {type:"string"}

In [50]:
# =====================================================
# GENERATE
# =====================================================

%%time
if MODE == "equation":
    gen_result = equation_generator.invoke({
        "domain": DOMAIN,
        "topic": TOPIC,
        "difficulty": DIFFICULTY,
        "target_language": LANGUAGE,
    })

    # Extract for solver
    item = gen_result["equation"]
    item_markdown = gen_result["equation_markdown"]
    item_translated = gen_result["equation_markdown_translated"]

    print("=== EQUATION ===")
    display(Markdown(item_markdown))
    if LANGUAGE and LANGUAGE.lower() != "english":
        print(f"\n=== EQUATION ({LANGUAGE}) ===")
        display(Markdown(item_translated))

else:  # problem
    gen_result = problem_generator.invoke({
        "domain": DOMAIN,
        "topic": TOPIC,
        "difficulty": DIFFICULTY,
        "target_language": LANGUAGE,
    })

    # Extract for solver
    item = gen_result["problem"]
    item_markdown = gen_result["problem_markdown"]
    item_translated = gen_result["problem_markdown_translated"]

    print("=== PROBLEM ===")
    display(Markdown(item_markdown))
    if LANGUAGE and LANGUAGE.lower() != "english":
        print(f"\n=== PROBLEM ({LANGUAGE}) ===")
        display(Markdown(item_translated))

# Print token usage
tu = gen_result.get("token_usage", {})
print(f"\n📊 Generation Tokens: {format_token_summary(tu)}")
print(f"Errors: {gen_result.get('errors', [])}")

=== PROBLEM ===


Construct a function $f:[0,1]\to\mathbb{R}$ from the quadratic family
$$f(x)=ax^2+bx+c$$
that satisfies the endpoint constraints $f(0)=1$ and $f(1)=2$, and has the additional constraint that its slope at the left endpoint is zero: $f'(0)=0$.

When the graph of $y=f(x)$ on $[0,1]$ is revolved about the $x$-axis, it generates a solid with volume
$$V=\pi\int_0^1 \big(f(x)\big)^2\,dx.$$

Determine the coefficients $a,b,c$ so that the resulting solid has prescribed volume
$$V=\frac{31\pi}{5}.$$

(Your constructed $f$ should satisfy all three constraints and the volume condition.)


=== PROBLEM (Ukrainian) ===


Побудуйте функцію $f:[0,1]\to\mathbb{R}$ із квадратичної родини
$$f(x)=ax^2+bx+c$$
яка задовольняє крайові умови $f(0)=1$ та $f(1)=2$, а також додаткову умову, що її похідна в лівій крайовій точці дорівнює нулю: $f'(0)=0$.

Коли графік $y=f(x)$ на $[0,1]$ обертається навколо осі $x$, він утворює тіло з об’ємом
$$V=\pi\int_0^1 \big(f(x)\big)^2\,dx.$$

Визначте коефіцієнти $a,b,c$ так, щоб отримане тіло мало заданий об’єм
$$V=\frac{31\pi}{5}.$$

(Побудована вами функція $f$ має задовольняти всі три умови та умову на об’єм.)


📊 Generation Tokens: Input: 925 | Output: 972 | Total: 1897
Errors: []
CPU times: user 484 ms, sys: 41.9 ms, total: 526 ms
Wall time: 14 s


## Solve the Problem

In [51]:
# =====================================================
# SOLVE
# =====================================================

%%time
solve_result = problem_solver.invoke({
    "problem": item,
    "target_language": LANGUAGE,
    "step_by_step": not QUICK_SOLUTION,
}, {"recursion_limit": 50})

# Print plan only if step-by-step
if not QUICK_SOLUTION:
    print("=== SOLUTION PLAN ===")
    for step in solve_result.get("solution_plan", []):
        step_desc = step['description'][:80] + "..." if len(step['description']) > 80 else step['description']
        print(f"  Step {step['step_number']} [{step['step_type']}]: {step_desc}")
        if step.get("wolfram_query"):
            print(f"    → WA: {step['wolfram_query']}")
    print()

# Print summary
tu = solve_result.get("token_usage", {})
print(f"📊 Solution Tokens: {format_token_summary(tu)}")
print(f"WA calls: {solve_result.get('wa_calls', 0)} | Errors: {len(solve_result.get('solution_errors', []))}")
if solve_result.get("solution_errors"):
    print(f"Solution errors: {solve_result['solution_errors']}")

=== SOLUTION PLAN ===
  Step 1 [reasoning]: Write f(x)=ax^2+bx+c and translate endpoint/slope constraints into equations: f(...
  Step 2 [symbolic_compute]: Compute f'(x) and apply f'(0)=0 to get an equation in a,b.
    → WA: Differentiate a x^2 + b x + c with respect to x
  Step 3 [reasoning]: Apply f(0)=1 to solve for c.
  Step 4 [symbolic_compute]: Apply f(1)=2 and the slope condition equation to solve for a and b (with c alrea...
    → WA: Solve[{a*1^2 + b*1 + 1 == 2, b == 0},{a,b}]
  Step 5 [reasoning]: Substitute a,b,c into f(x) to get the explicit quadratic.
  Step 6 [symbolic_compute]: Set up the volume condition V=pi*Integrate[(f(x))^2,{x,0,1}] and compute the int...
    → WA: Integrate[(a x^2 + b x + c)^2,{x,0,1}]
  Step 7 [symbolic_compute]: Impose the prescribed volume by setting the computed integral equal to 31/5 and ...
    → WA: Solve[Integrate[(a x^2 + b x + c)^2,{x,0,1}] == 31/5 && c==1 && b==0 && a==1, {a,b,c}]
  Step 8 [verify]: Verify: check f(0)=1, f(1)=2, f'(0)=0

In [52]:
# =====================================================
# DISPLAY SOLUTION
# =====================================================

print("=== SOLUTION ===")
print_solution(solve_result["solution_markdown"])

if LANGUAGE and LANGUAGE.lower() != "english":
    print(f"\n=== SOLUTION ({LANGUAGE}) ===")
    print_solution(solve_result["solution_markdown_translated"])

=== SOLUTION ===


## Solution

Let
$$
f(x)=ax^2+bx+c,\qquad x\in[0,1].
$$

### 1) Impose the endpoint constraints $f(0)=1$ and $f(1)=2$

From $f(0)=1$:
$$
f(0)=c=1 \quad\Longrightarrow\quad c=1.
$$

From $f(1)=2$:
$$
f(1)=a+b+c=2.
$$
Substitute $c=1$:
$$
a+b+1=2 \quad\Longrightarrow\quad a+b=1.
$$

So far,
$$
c=1,\qquad a+b=1.
$$

### 2) Impose the slope constraint $f'(0)=0$

Differentiate:
$$
f'(x)=2ax+b.
$$
Then
$$
f'(0)=b=0 \quad\Longrightarrow\quad b=0.
$$

Substitute $b=0$ into $a+b=1$:
$$
a=1.
$$

Thus the three constraints $f(0)=1$, $f(1)=2$, $f'(0)=0$ force a *unique* quadratic:
$$
(a,b,c)=(1,0,1),\qquad f(x)=x^2+1.
$$

### 3) Compute the resulting volume and compare with the prescribed value

The volume of the solid of revolution about the $x$-axis is
$$
V=\pi\int_0^1 (f(x))^2\,dx=\pi\int_0^1 (x^2+1)^2\,dx.
$$
Expand:
$$
(x^2+1)^2=x^4+2x^2+1.
$$
Integrate term-by-term:
$$
\int_0^1 (x^4+2x^2+1)\,dx
=\left[\frac{x^5}{5}\right]_0^1+2\left[\frac{x^3}{3}\right]_0^1+\left[x\right]_0^1
=\frac15+\frac23+1=\frac{28}{15}.
$$
Hence
$$
V=\pi\cdot\frac{28}{15}=\frac{28\pi}{15}.
$$

But the prescribed volume is
$$
\frac{31\pi}{5}.
$$
Since
$$
\frac{28\pi}{15}\neq \frac{31\pi}{5},
$$
the volume condition is incompatible with the three point/slope constraints.

---

## Final Answer

The constraints $f(0)=1$, $f(1)=2$, and $f'(0)=0$ uniquely determine
$$
\boxed{(a,b,c)=(1,0,1)} \quad\text{so}\quad \boxed{f(x)=x^2+1}.
$$
This function generates volume
$$
\boxed{V=\frac{28\pi}{15}},
$$
so **no quadratic** $ax^2+bx+c$ can satisfy *all* three constraints **and** the prescribed volume $V=\dfrac{31\pi}{5}$ simultaneously.

---

> **Note:** This solution may contain errors or incomplete reasoning. The following issues were detected:
> - Solution lacks mathematical notation.
>
> Please verify the steps independently.


=== SOLUTION (Ukrainian) ===


## Розв’язання

Нехай
$$
f(x)=ax^2+bx+c,\qquad x\in[0,1].
$$

### 1) Накладімо крайові умови $f(0)=1$ і $f(1)=2$

Із $f(0)=1$:
$$
f(0)=c=1 \quad\Longrightarrow\quad c=1.
$$

Із $f(1)=2$:
$$
f(1)=a+b+c=2.
$$
Підставимо $c=1$:
$$
a+b+1=2 \quad\Longrightarrow\quad a+b=1.
$$

Отже, наразі
$$
c=1,\qquad a+b=1.
$$

### 2) Накладімо умову на похідну $f'(0)=0$

Продиференціюймо:
$$
f'(x)=2ax+b.
$$
Тоді
$$
f'(0)=b=0 \quad\Longrightarrow\quad b=0.
$$

Підставимо $b=0$ у $a+b=1$:
$$
a=1.
$$

Отже, три умови $f(0)=1$, $f(1)=2$, $f'(0)=0$ задають *єдиний* квадратний многочлен:
$$
(a,b,c)=(1,0,1),\qquad f(x)=x^2+1.
$$

### 3) Обчислімо отриманий об’єм і порівняймо із заданим значенням

Об’єм тіла обертання навколо осі $x$ дорівнює
$$
V=\pi\int_0^1 (f(x))^2\,dx=\pi\int_0^1 (x^2+1)^2\,dx.
$$
Розкриємо дужки:
$$
(x^2+1)^2=x^4+2x^2+1.
$$
Проінтегруймо почленно:
$$
\int_0^1 (x^4+2x^2+1)\,dx
=\left[\frac{x^5}{5}\right]_0^1+2\left[\frac{x^3}{3}\right]_0^1+\left[x\right]_0^1
=\frac15+\frac23+1=\frac{28}{15}.
$$
Отже,
$$
V=\pi\cdot\frac{28}{15}=\frac{28\pi}{15}.
$$

Але заданий об’єм дорівнює
$$
\frac{31\pi}{5}.
$$
Оскільки
$$
\frac{28\pi}{15}\neq \frac{31\pi}{5},
$$
умова на об’єм несумісна з трьома умовами на значення/нахил у точках.

---

## Остаточна відповідь

Умови $f(0)=1$, $f(1)=2$ та $f'(0)=0$ однозначно визначають
$$
\boxed{(a,b,c)=(1,0,1)} \quad\text{тобто}\quad \boxed{f(x)=x^2+1}.
$$
Ця функція породжує об’єм
$$
\boxed{V=\frac{28\pi}{15}},
$$
тому **жоден квадратний многочлен** $ax^2+bx+c$ не може одночасно задовольняти *всі* три умови **та** заданий об’єм $V=\dfrac{31\pi}{5}$.

---

> **Примітка:** Це розв’язання може містити помилки або неповні міркування. Було виявлено такі проблеми:
> - Розв’язання не містить математичних позначень.
>
> Будь ласка, перевірте кроки самостійно.

In [53]:
# =====================================================
# COMBINED SUMMARY
# =====================================================

gen_tokens = gen_result.get("token_usage", {})
solve_tokens = solve_result.get("token_usage", {})

total_tokens = {
    "input": gen_tokens.get("input", 0) + solve_tokens.get("input", 0),
    "output": gen_tokens.get("output", 0) + solve_tokens.get("output", 0),
    "total": gen_tokens.get("total", 0) + solve_tokens.get("total", 0),
}

print("=" * 50)
print("SUMMARY")
print("=" * 50)
print(f"Mode: {MODE} | Step-by-step: {not QUICK_SOLUTION}")
print(f"Domain: {DOMAIN} | Topic: {TOPIC} | Difficulty: {DIFFICULTY}")
print(f"Language: {LANGUAGE or 'English'}")
print()
print(f"Generation Tokens: {format_token_summary(gen_tokens)}")
print(f"Solution Tokens:   {format_token_summary(solve_tokens)}")
print(f"TOTAL Tokens:      {format_token_summary(total_tokens)}")
print(f"WA Calls: {solve_result.get('wa_calls', 0)}")

SUMMARY
Mode: problem | Step-by-step: True
Domain: calculus | Topic: Volume Calculation | Difficulty: medium
Language: Ukrainian

Generation Tokens: Input: 925 | Output: 972 | Total: 1897
Solution Tokens:   Input: 33514 | Output: 11499 | Total: 45013
TOTAL Tokens:      Input: 34439 | Output: 12471 | Total: 46910
WA Calls: 11
